In [1]:
from numpy import *
from scipy.sparse import *
import implicit.

In [2]:
playlists=genfromtxt("train_final.csv",skip_header=1,usecols=0)
playlists

array([  3271849.,   5616275.,  11267488., ...,   2659739.,  10687820.,
         4962329.])

In [128]:
tracks=genfromtxt("train_final.csv",skip_header=1,usecols=1)
tracks

array([ 2801526.,   727878.,  2805283., ...,  1468602.,  1187581.,
        1094523.])

In [4]:
playlists.shape,tracks.shape

((1040522,), (1040522,))

In [5]:
data=ones(playlists.shape)
data

array([ 1.,  1.,  1., ...,  1.,  1.,  1.])

In [120]:
# MAGIA_1 = creare velocemente la URM
URM=coo_matrix((data,(tracks,playlists)))
URM

<5018275x11766363 sparse matrix of type '<class 'numpy.float64'>'
	with 1040522 stored elements in COOrdinate format>

In [7]:
URM=csc_matrix(URM).astype(int32)

In [8]:
max(tracks)

5018274.0

In [9]:
max(playlists)

11766362.0

In [10]:
URM

<11766363x5018275 sparse matrix of type '<class 'numpy.int32'>'
	with 1040522 stored elements in Compressed Sparse Column format>

In [11]:
P=URM*URM.T

In [13]:
P

<11766363x11766363 sparse matrix of type '<class 'numpy.int32'>'
	with 28025251 stored elements in Compressed Sparse Column format>

In [19]:
M=URM

In [21]:
# MAGIA_2 = creare le reduced velocemente
M = M[M.getnnz(1)>0][:,M.getnnz(0)>0]
M

<45649x99999 sparse matrix of type '<class 'numpy.int32'>'
	with 1040522 stored elements in Compressed Sparse Column format>

In [27]:
t99999=unique(tracks.astype(int32))

In [29]:
t100k=load("ordered_tracks.npy")

In [31]:
#questa track non compare in nessuna playlist
setdiff1d(t100k,t99999)

array([ 3626362.])

In [37]:
from implicit.als import AlternatingLeastSquares
#from implicit.approximate_als import (AnnoyAlternatingLeastSquares, NMSLibAlternatingLeastSquares,FaissAlternatingLeastSquares)
from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,
                                         TFIDFRecommender, bm25_weight)

In [39]:
MODELS = {"als":  AlternatingLeastSquares,
          "tfidf": TFIDFRecommender,
          "cosine": CosineRecommender,
          "bm25": BM25Recommender}

In [41]:
def get_model(model_name):
    model_class = MODELS.get(model_name)
    if not model_class:
        raise ValueError("Unknown Model '%s'" % model_name)

    # some default params
    if issubclass(model_class, AlternatingLeastSquares):
        params = {'factors': 50, 'dtype': numpy.float32}
    elif model_name == "bm25":
        params = {'K1': 100, 'B': 0.5}
    else:
        params = {}

    return model_class(**params)

In [121]:
modelCos=get_model("cosine")
modelTFIDF=get_model("tfidf")

In [122]:
modelCos.fit(M)
modelTFIDF.fit(M)

In [93]:
simCos=modelCos.similarity
simTFIDF=modelTFIDF.similarity
simCos,simTFIDF

(<45649x45649 sparse matrix of type '<class 'numpy.float64'>'
 	with 861653 stored elements in Compressed Sparse Row format>,
 <45649x45649 sparse matrix of type '<class 'numpy.float64'>'
 	with 861653 stored elements in Compressed Sparse Row format>)

In [94]:
simCos.setdiag(0)
simTFIDF.setdiag(0)

In [91]:
#cosine
argsort(simCos[0].todense())

matrix([[    0, 30423, 30424, ...,  4796, 44772, 19688]])

In [111]:
#tfidf
argsort(simTFIDF[0].todense())

matrix([[    0, 30422, 30423, ..., 38467, 44772, 19688]])

In [126]:
tracks.size

1040522

In [123]:
## RIFARE usando panda
pTracks=URM.T.tocsr()
with open("provalibreria","w") as o:
    for pID,playlistname in enumerate(playlists):
        for trackID,score in modelCos.recommend(pID,pTracks):
            o.write("%s\t%s\t%s\n" % (playlistname, tracks[trackID], score))

ValueError: dimension mismatch